# Reconocimiento de patrones: Identificación de grupos o Clustering
### Ramón Soto C. [(rsotoc@moviquest.com)](mailto:rsotoc@moviquest.com/)
![ ](images/blank.png)
![agents](images/binary_data_under_a_magnifying.jpg)
[ver en nbviewer](http://nbviewer.ipython.org/github/rsotoc/pattern-recognition/blob/master/Clustering%20V.ipynb)

## Técnicas de clustering: *ISODATA*

La técnica de *k-medias* (o *k-means*) es una de las técnicas de clustering más simples y más utilizadas; es una técnica muy simple y rápida. Sin embargo, esta técnica sufre de las siguientes limitantes importantes: a) escala pobre computacionalmente, b) su eficacia es fuertemente dependiente de la selección de $k$ c) la elección de los centroide siniciales puede provocar que el método caiga en mínimos locales y d) el valor de $k$ debe ser proporcionado por el usuario y no puede modificarse una vez que inicia el proceso. 

El método **ISODATA** (*Iterative Self-Organizing Data Analysis Techniques*) es un algoritmo similar al $k$-means; su objetivo es particionar un conjunto de datos en subconjuntos. Sin embargo, a diferencia de $k$-means, el método Isodata maneja una serie de [*heurísticas*](https://en.wikipedia.org/wiki/Heuristic) con tres objetivos que le permiten optimizar el número de clusters:

* Eliminar clusters con poco ejemplares.
* Unir clusters muy cercanos.
* Dividir clusters dispersos

![ ](images/isodata-1.png)


### Parámetros

El algoritmo Isodata utiliza los siguientes parámetros:

* $k_{init}$: Número inicial de clusters.
* $k$: el número actual de clusters
* $n_{min}$: Mínimo número de elementos en un cluster.
* $I_{max}$: Máximo número de iteraciones.
* $\sigma_{max}$: Máximo valor de la desviación estándar de los puntos al centro de su cluster, a lo largo de cada eje.
* $L_{min}$: Distancia mínima entre los centroides de las clases.
* $P_{max}$: Número máximo de clusters que pueden ser unificados en una iteración.


### Algoritmo

El algoritmo Isodata sigue los siguientes pasos (dado un conjunto de datos):
<ol>
<li> Definir los valores de $k_{init}, n_{min}, I_{max}, \sigma_{max}, L_{min}$ y $P_{max}$.</li>
<li style="padding-top:4pt"> Seleccionar arbitrariamente los centroides iniciales. </li>
<li style="padding-top:4pt"> Asignar cada punto del conjunto de datos al cluster donde la distancia del punto al centroide es menor. </li>
<li style="padding-top:4pt"> Eliminar los clusters con menos de $n_{min}$ elementos. Ajustar el valor de $k$ y reetiquetar los clusters.</li>
<li style="padding-top:4pt"> Recalcular los centroides a partir de los puntos actualmente en cada cluster. Si se eliminaron clusters en el paso 4) el algoritmo regresa al paso 3).</li>
<li style="padding-top:4pt"> Calcular las distancias promedio $\Delta_j$ de los puntos de un cluster a su centroide y la distancia promedio general (de cada punto al centroide más cercano) $\Delta$.</li>
<li style="padding-top:4pt"> Si esta es la última iteración, terminar. En caso contrario verificar si quedan la mitad o menos de los clusters iniciales y de ser así ir al paso 8 (dividir clusters). En caso contrario, si la iteración es par o el número de clusters es mayor que el doble de los clusters iniciales, entonces ir al paso 9 (unir). En caso contrario, volver al paso 3 (como $k$-means).</li>
<li style="padding-top:4pt"> Calcular, para cada cluster $S_j$ un vector $\mathbf{s_j} = (s_{j1}\ldots s_{jn_j})$ con las desviaciones estándar $s_{ji}$ de cada atributo $i$ de los elementos $\mathbf{x} = (x_{1}\ldots x_{n_j})$ en el cluster (con respecto al centroide $\mathbf{z}_j$).
$$s_{ji}=\frac{1}{n_j}\sqrt{\sum_{\mathbf{x}\in S_j} (x_i-z_{ji})^2 }$$
Si al menos una componente del vector de desviaciones estándar sobrepasa la máxima desviación estándar permitida, $\sigma_{max}$ consideramos dividir la clase, siempre que se cumpla alguna de las siguientes condiciones adicionales:
<ul>
<li> Que la distancia promedio entre puntos en el cluster sea mayor que la distancia promedio global y que el cluster tenga más del doble de elementos que el mínimo permitido para un cluster ($n_{min}$), o</li>
<li> Que el número actual de clusters sea menor que la mitad del número de clusters inicial.</li>
</ul><br>

Si se cumple alguna de estas condiciones, reemplazar el cluster por dos nuevos clusters. Seleccionar como centroides de estos clusters los dos vectores más alejados entre sí en el cluster original.

Si hubo al menos una división de clusters, volver a reconstruir todos los clusters (y centroides).</li>

<li style="padding-top:4pt"> Calcular las distancias *inter-clusters* entre todos los centroides de los clusters. Seleccionar el máximo número permitido de pares de clusters para ser unificados ($P_{max}$), que satisfagan las siguientes condiciones:
<ul>
<li> Que la distancia entre los centroides sea menor que la mínima distancia permitida ($L_{min}$), y</li>
<li>Que ninguno de los dos clusters haya participado en una unificación en la presente iteración.</li>
</ul><br>
Si se cumplen estas condiciones, reemplazar los dos cluster por un nuevo cluster cuyo centroide es el punto medio de los centroides originales.

Si hubo al menos una unificación de clusters, volver a reconstruir todos los clusters.
</li>
</ol>

In [1]:
# Inicializar el ambiente
import numpy as np
import pandas as pd
import math
import random
import time
import sys

from IPython.display import display, HTML
from scipy.spatial.distance import euclidean, pdist, squareform
np.set_printoptions(precision=2, suppress=True) # Cortar la impresión de decimales a 1

LARGER_DISTANCE = sys.maxsize
TALK = True # TALK = True, imprime resultados parciales

In [2]:
# Leer los datos de archivo
df = pd.read_csv("Data sets/datosProm.csv", names = ['A', 'B'])

# Agregar una columna "cluster" inicializada a null 
df["Cluster"] = np.nan
DATA_LEN = df.shape[0]

print (df.describe())

                A          B  Cluster
count   30.000000  30.000000      0.0
mean    62.576333  62.595633      NaN
std     35.940813  29.057323      NaN
min      0.000000   0.000000      NaN
25%     68.720000  57.262500      NaN
50%     77.345000  75.655000      NaN
75%     86.000000  83.262750      NaN
max    100.000000  95.063000      NaN


1) Definir los valores de $k_{init}, n_{min}, I_{max}, \sigma_{max}, L_{min}$ y $P_{max}$:

In [3]:
K_INIT = 5
N_MIN = 3
I_MAX = 10
S_MAX = 5
L_MIN = 80
P_MAX = 2

num_clusters = K_INIT # valor de k
iteration = 0

2) Seleccionar arbitrariamente los centroides iniciales 

In [4]:
# Definir forma de muestreo; 0 = random, 1=head, 2=tail
SAMPLING_METHOD = 1 

def initialize_centroids():
    global centroids
    if (TALK) : 
        print("Centroides inicializados en:")
        
    if (SAMPLING_METHOD == 0) :
        centroids = df.sample(n=num_clusters).reset_index(drop=True)
    elif (SAMPLING_METHOD == 1):
        centroids = df.head(n=num_clusters).reset_index(drop=True)
    else :
        centroids = df.tail(n=num_clusters).reset_index(drop=True)
    centroids.drop('Cluster', axis=1, inplace=True)
    
    if (TALK) : 
        display(centroids)        
        print()
    
    return

# --------------------------
# Inicializar los centroides
initialize_centroids()

Centroides inicializados en:


,A,B
0,70.28,42.125
1,0.00,56.750
2,79.00,2.500
3,75.64,11.667
4,82.00,58.800


3) Asignar cada punto del conjunto de datos al cluster donde la distancia del punto al centroide es menor. 

4) Eliminar los clusters con menos de $n_{min}$ elementos. Ajustar el valor de $k$ y reetiquetar los clusters.

In [5]:
elim = False
members = []
def update_clusters():
    global num_clusters, elim, members, centroids
    changed = False
    cluster_col_index = df.shape[1] - 1
    
    if (TALK) :
        print("Actualizando clusters")
    for index, row in df.iterrows():
        minDistance = LARGER_DISTANCE
        currentCluster = 0
        
        for j, rc in centroids.iterrows():
            dist = euclidean(row[['A', 'B']], rc)
            if(dist < minDistance):
                minDistance = dist
                currentCluster = j
        
        if(pd.isnull(row['Cluster']) or row['Cluster'] != currentCluster):
            df.iloc[index, cluster_col_index] = currentCluster
            changed = True
            
    members = [0] * num_clusters
    for i in range(num_clusters):
        members[i] = df[df["Cluster"]==i].count()["Cluster"]
        if (TALK) : 
            print("El cluster ", i, " incluye ", members[i], "miembros.")
    if (TALK) : 
        print()

    # Marcar los grupos a eliminar      
    to_eliminate = []
    for j in range(num_clusters):
        if (members[j] < N_MIN):
            to_eliminate.append(j)
    if len(to_eliminate) > 0:
        elim = True
        print("Clusters a eliminar:", to_eliminate)
        # Eliminar los centroides seleccionados
        centroids.drop(to_eliminate, inplace=True)    
        centroids = centroids.reset_index(drop=True)
        num_clusters = centroids.shape[0]
        changed = True
    else :
        elim = False
    
    if changed:
        for index, row in df.iterrows():
            minDistance = LARGER_DISTANCE
            currentCluster = 0

            for j, rc in centroids.iterrows():
                dist = euclidean(row[['A', 'B']], rc)
                if(dist < minDistance):
                    minDistance = dist
                    currentCluster = j

            if(pd.isnull(row['Cluster']) or row['Cluster'] != currentCluster):
                df.iloc[index, cluster_col_index] = currentCluster
                changed = True
    
    
    display(centroids, df)
    
    return changed

# --------------------------
# Actualizar los clusters
KEEP_WALKING = update_clusters()

Actualizando clusters
El cluster  0  incluye  2 miembros.
El cluster  1  incluye  7 miembros.
El cluster  2  incluye  3 miembros.
El cluster  3  incluye  2 miembros.
El cluster  4  incluye  16 miembros.

Clusters a eliminar: [0, 3]


,A,B
0,0.0,56.75
1,79.0,2.50
2,82.0,58.80


,A,B,Cluster
0,70.28,42.125,2.0
1,0.00,56.750,0.0
2,79.00,2.500,1.0
3,75.64,11.667,1.0
4,82.00,58.800,2.0
5,86.00,77.265,2.0
6,80.00,85.500,2.0
7,68.20,62.440,2.0
8,72.00,88.000,2.0
9,74.00,80.604,2.0


5) Recalcular los centroides a partir de los puntos actualmente en cada cluster. Si se eliminaron clusters en el paso 4) el algoritmo regresa al paso 3).

In [6]:
def update_centroids():
    global centroids
        
    for j, rc in centroids.iterrows():
        centroids.iloc[j] = df[df["Cluster"]==j].mean()[['A', 'B']]

    if (TALK) : 
        print("Los nuevos centroids son:\n", centroids)        
        print()
    
    return

# --------------------------
# Actualizar los centroides
update_centroids()
if (elim) :
    KEEP_WALKING = update_clusters()
    update_centroids()

Los nuevos centroids son:
            A          B
0   0.000000  73.886571
1  84.010000   6.466800
2  80.957778  73.796056

Actualizando clusters
El cluster  0  incluye  7 miembros.
El cluster  1  incluye  6 miembros.
El cluster  2  incluye  17 miembros.



,A,B
0,0.000000,73.886571
1,84.010000,6.466800
2,80.957778,73.796056


,A,B,Cluster
0,70.28,42.125,2.0
1,0.00,56.750,0.0
2,79.00,2.500,1.0
3,75.64,11.667,1.0
4,82.00,58.800,2.0
5,86.00,77.265,2.0
6,80.00,85.500,2.0
7,68.20,62.440,2.0
8,72.00,88.000,2.0
9,74.00,80.604,2.0


Los nuevos centroids son:
            A          B
0   0.000000  73.886571
1  81.808333  11.222333
2  81.555294  76.078176



6) Calcular las distancias promedio $\Delta_j$ de los puntos de un cluster a su centroide y la distancia promedio general (de cada punto al centroide más cercano) $\Delta$.

In [7]:
deltas = []
delta = 0
def update_deltas():
    global deltas, delta, centroids
    deltas = [0] * num_clusters
    N = 0
    cluster = 0
    for _, rc in centroids.iterrows():
        n = 0
        for i, row in df[df["Cluster"]==cluster].iterrows():
            deltas[cluster] += euclidean(row[['A', 'B']], rc)
            n += 1
        delta += deltas[cluster]
        deltas[cluster] /= n
        N += n
        cluster += 1
    delta /= N
    
    if (TALK) : 
        print("Las distancias medias en cada cluster son:\n", deltas)   
        print("\nLa distancia media promedio es:", delta)   
        
    return
    
update_deltas()

Las distancias medias en cada cluster son:
 [8.882897959183678, 12.762142305287727, 13.744777384565223]

La distancia media promedio es: 12.413811836120695


7) Si esta es la última iteración, terminar. En caso contrario verificar si quedan la mitad o menos de los clusters iniciales y de ser así ir al paso 8 (dividir clusters). En caso contrario, si la iteración es par o el número de clusters es mayor que el doble de los clusters iniciales, entonces ir al paso 9 (unir). En caso contrario, volver al paso 3 (como $k$-means).

In [8]:
# Ejecutar sólo desués de haber "activado" los pasos 8 y 9
#while(iteration < I_MAX and KEEP_WALKING) :
#    if (num_clusters <= K_INIT / 2) :
#        divide_clusters()
#    elif (iteration % 2 == 0 or num_clusters > 2 * K_INIT) :
#        mix_clusters()
#        
#    KEEP_WALKING = update_clusters()
#    if (KEEP_WALKING):
#        update_centroids()
#    else :
#        if (TALK) : 
#            print ("No más cambios.")
#    iteration += 1

8) Calcular, para cada cluster $S_j$ un vector $\mathbf{s_j} = (s_{j1}\ldots s_{jn_j})$ con las desviaciones estándar $s_{ji}$ de cada atributo $i$ de los elementos $\mathbf{x} = (x_{1}\ldots x_{n_j})$ en el cluster (con respecto al centroide $\mathbf{z}_j$).

$$s_{ji}=\frac{1}{n_j}\sqrt{\sum_{\mathbf{x}\in S_j} (x_i-z_{ji})^2 }$$

Si al menos una componente del vector de desviaciones estándar sobrepasa la máxima desviación estándar permitida, $\sigma_{max}$ consideramos dividir la clase, siempre que se cumpla alguna de las siguientes condiciones adicionales:

* Que la distancia promedio entre puntos en el cluster sea mayor que la distancia promedio global y que el cluster tenga más del doble de elementos que el mínimo permitido para un cluster ($n_{min}$), o

* Que el número actual de clusters sea menor que la mitad del número de clusters inicial.

Si se cumple alguna de estas condiciones, reemplazar el cluster por dos nuevos clusters. Seleccionar como centroides de estos clusters los dos vectores más alejados entre sí en el cluster original.

Si hubo al menos una división de clusters, volver a reconstruir todos los clusters (y centroides).

In [9]:
def divide_clusters():
    global num_clusters, centroids

    display(centroids)
    
    # Cálculo de desviaciones estandar
    sigma_vect = []
    for c in range(num_clusters):
        sigma_vect.append(list(df[df["Cluster"]==c].std()[['A', 'B']]))    
    display(sigma_vect)
    
    candidates = []
    for c in range(num_clusters):
        for i in range(df.shape[1] - 1):
            if (sigma_vect[c][i] > S_MAX):
                candidates.append(c)
                break # Sucio... pero eficiente :-) ... ya encontramos un atributo con elevada sigma

    if (TALK) :
        print("Posibles clusters a dividir:", candidates)
    
    divided = False
    to_eliminate = []
    for c in candidates:
        cond = num_clusters < K_INIT/2 or (deltas[c] > delta and members[c] > 2 * N_MIN)
        if cond:
            dist_matrix = squareform(pdist(df[df["Cluster"]==c], 'euclidean'))
            idx = (dist_matrix==dist_matrix.max()).argmax()
            z1 = idx // members[c]
            z2 = idx % members[c]
            if (TALK) :
                print("\nSe dividirá el cluster {}.".format(c))
                print("Se crearán nuevos clusters en torno a {} y {}."
                     .format(z1, z2))
            to_eliminate.append(c)
            centroids = centroids.append(df.iloc[z1][['A', 'B']], 
                                         ignore_index=True, sort=False)
            centroids = centroids.append(df.iloc[z2][['A', 'B']], 
                                         ignore_index=True, sort=False)
            num_clusters += 1
            

    if len(to_eliminate) > 0 :
        centroids.drop(to_eliminate, inplace=True)
        centroids = centroids.reset_index(drop=True)

        if (TALK) : 
            display(centroids)
            print("")
            
        update_clusters()
        update_centroids()
    
    return 

divide_clusters()

,A,B
0,0.000000,73.886571
1,81.808333,11.222333
2,81.555294,76.078176


[[0.0, 11.802058815550541],
 [9.835890232544621, 12.72866202971336],
 [8.74492003797566, 13.742614353877931]]

Posibles clusters a dividir: [0, 1, 2]

Se dividirá el cluster 2.
Se crearán nuevos clusters en torno a 0 y 10.


,A,B
0,0.000000,73.886571
1,81.808333,11.222333
2,70.280000,42.125000
3,91.720000,0.000000



Actualizando clusters
El cluster  0  incluye  7 miembros.
El cluster  1  incluye  3 miembros.
El cluster  2  incluye  18 miembros.
El cluster  3  incluye  2 miembros.

Clusters a eliminar: [3]


,A,B
0,0.000000,73.886571
1,81.808333,11.222333
2,70.280000,42.125000


,A,B,Cluster
0,70.28,42.125,2.0
1,0.00,56.750,0.0
2,79.00,2.500,1.0
3,75.64,11.667,1.0
4,82.00,58.800,2.0
5,86.00,77.265,2.0
6,80.00,85.500,2.0
7,68.20,62.440,2.0
8,72.00,88.000,2.0
9,74.00,80.604,2.0


Los nuevos centroids son:
            A          B
0   0.000000  73.886571
1  84.010000   6.466800
2  80.957778  73.796056



9) Calcular las distancias *inter-clusters* entre todos los centroides de los clusters. Seleccionar el máximo número permitido de pares de clusters para ser unificados ($P_{max}$), que satisfagan las siguientes condiciones:

* Que la distancia entre los centroides sea menor que la mínima distancia permitida ($L_{min}$), y

* Que ninguno de los dos clusters haya participado en una unificación en la presente iteración.

Si se cumplen estas condiciones, reemplazar los dos cluster por un nuevo cluster cuyo centroide es el punto medio de los centroides originales.

Si hubo al menos una unificación de clusters, volver a reconstruir todos los clusters.

In [10]:
def mix_clusters():
    global centroids, num_clusters
    dist_matrix = np.triu(squareform(pdist(centroids, 'euclidean')))
    
    flag = math.floor(dist_matrix.max() * 10)
    dist_matrix[dist_matrix == 0] = flag
    
    mixed = False
    to_eliminate = []
    while dist_matrix.min() < flag:
        dist_min = dist_matrix.min()
        idx = (dist_matrix==dist_min).argmax()
        z1 = idx // len(centroids)
        z2 = idx % len(centroids)        

        if dist_min < L_MIN:
            display(centroids)
            z = [sum(x)/2 for x in zip(centroids.iloc[z1], centroids.iloc[z2])]
            centroids.iloc[z1] = z
            to_eliminate.append(z2)
            num_clusters -= 1
            mixed = True
            if(TALK):
                print("Unificando clusters {} y {}.\nSe creará nuevo centroide en {}\n"
                      .format(z1, z2, z))

        dist_matrix[z1][z2] = flag
            
    centroids.drop(to_eliminate, inplace=True)
    centroids = centroids.reset_index(drop=True)
    
    if (mixed) :
        update_clusters()
        update_centroids()

    return

mix_clusters()

,A,B
0,0.000000,73.886571
1,84.010000,6.466800
2,80.957778,73.796056


Unificando clusters 1 y 2.
Se creará nuevo centroide en [82.48388888888888, 40.13142777777777]

Actualizando clusters
El cluster  0  incluye  7 miembros.
El cluster  1  incluye  23 miembros.



,A,B
0,0.000000,73.886571
1,82.483889,40.131428


,A,B,Cluster
0,70.28,42.125,1.0
1,0.00,56.750,0.0
2,79.00,2.500,1.0
3,75.64,11.667,1.0
4,82.00,58.800,1.0
5,86.00,77.265,1.0
6,80.00,85.500,1.0
7,68.20,62.440,1.0
8,72.00,88.000,1.0
9,74.00,80.604,1.0


Los nuevos centroids son:
            A          B
0   0.000000  73.886571
1  81.621304  59.159261



In [11]:
# Reproducido aquí para facilitar la ejecución
iteration +=1
while iteration < I_MAX and KEEP_WALKING :
    if num_clusters <= K_INIT / 2 :
        divide_clusters()
    elif (iteration % 2 == 0 or num_clusters > 2 * K_INIT) :
        mix_clusters()
        
    KEEP_WALKING = update_clusters()
    if KEEP_WALKING:
        update_centroids()
        while elim :
            KEEP_WALKING = update_clusters()
            update_centroids()
    else :
        if TALK : 
            print ("No más cambios.")
    iteration += 1

,A,B
0,0.000000,73.886571
1,81.621304,59.159261


[[0.0, 11.802058815550541], [8.810081159644655, 31.97000333440053]]

Posibles clusters a dividir: [0, 1]

Se dividirá el cluster 0.
Se crearán nuevos clusters en torno a 0 y 4.

Se dividirá el cluster 1.
Se crearán nuevos clusters en torno a 9 y 15.


,A,B
0,70.28,42.125
1,82.00,58.800
2,74.00,80.604
3,0.00,59.933



Actualizando clusters
El cluster  0  incluye  7 miembros.
El cluster  1  incluye  4 miembros.
El cluster  2  incluye  12 miembros.
El cluster  3  incluye  7 miembros.



...

,A,B,Cluster
0,70.28,42.125,0.0
1,0.00,56.750,1.0
2,79.00,2.500,0.0
3,75.64,11.667,0.0
4,82.00,58.800,0.0
5,86.00,77.265,0.0
6,80.00,85.500,0.0
7,68.20,62.440,0.0
8,72.00,88.000,0.0
9,74.00,80.604,0.0


Los nuevos centroids son:
            A          B
0  81.621304  59.159261
1   0.000000  73.886571

Actualizando clusters
El cluster  0  incluye  23 miembros.



,A,B
0,81.621304,59.159261
1,0.000000,73.886571


,A,B,Cluster
0,70.28,42.125,0.0
1,0.00,56.750,1.0
2,79.00,2.500,0.0
3,75.64,11.667,0.0
4,82.00,58.800,0.0
5,86.00,77.265,0.0
6,80.00,85.500,0.0
7,68.20,62.440,0.0
8,72.00,88.000,0.0
9,74.00,80.604,0.0


No más cambios.


El método ISODATA es un método poco utilizado fuera del área de procesamiento de imágenes. Es difícil de configurar, por la cantidad y naturaleza de los parámetros a definir.